In [42]:
import pandas as pd
import numpy as np
from fredapi import Fred
from config import key, pwrd
from tqdm import tqdm, tqdm_notebook
import warnings
import time
from sqlalchemy import create_engine
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [30]:
#SQL connect to local database
# rds_connection_string = "postgres:"+ pwrd + "@localhost:5432/astoria"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [3]:
#define FRED API call
fred = Fred(api_key= key)

In [32]:
#set column view options
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 50)

In [33]:
# Code to search FRED database
# search = fred.search('500', order_by='popularity', sort_order='desc')
# search_df = pd.DataFrame(search[['title', 'id', 'popularity', 'units','seasonal_adjustment_short']])
# search_df[:20]

In [34]:
#desired indicators
indicators = ['USSLIND', 'UMCSENT', 'CSCICP03USM665S', 'INDPRO', 'TCU','DGORDER', 'NEWORDER', 'PI','BUSINV', 'TOTALSA',
             'MARTSMPCSM44X72USS', 'PAYEMS', 'UNRATE', 'ICSA', 'CCSA', 'CC4WSA', 'NPPTTL', 'CES0500000003', 'HSN1F',
             'EXHOSLUSM495S', 'PERMIT', 'CSUSHPINSA', 'SP500']

In [35]:
len(indicators)

23

In [38]:
#function to create dataframe and csv's for each indicator
errors=[]
for indicator in tqdm_notebook(indicators):
    try:
         #get data and info series
        data = fred.get_series(indicator, observation_start='1/31/1990')
        info = fred.get_series_info(indicator)
        #store as df's and reset indexes 
        data_df = pd.DataFrame(data)
        data_df.reset_index(level=0, inplace=True)
        info_df = pd.DataFrame(info)
        info_df.reset_index(level=0, inplace=True)
        #pivot info_df
        df_dict = dict(zip(info_df['index'],info_df[0]))
        info_df = pd.DataFrame(df_dict,  index=[0])
        #duplicate info_df rows to equal data_df length
        info_df = pd.concat([info_df]*(len(data_df)), ignore_index=True)
        #join df's
        combine_df = data_df.join(info_df)
        #rename columns
        combine_df.rename(columns = {'index':'date', 0:'data'}, inplace = True) 
        #create new 'change from last column'
        combine_df.insert(2, 'change', np.nan)
        #fill nan with zeroes
        combine_df['change'].fillna(0, inplace=True)
        #create data for 'change' column
        changes = [combine_df.iloc[i,1]-combine_df.iloc[i-1,1] for i in range (1,len(combine_df))]
        change_clean = [float("{0:.3f}".format(x)) for x in changes]
        combine_df['change'][1:len(combine_df)] = change_clean
        #write to csv
        combine_df.to_csv(f"data/{indicator}.csv",index = False, header = True)
        #export data to SQL
#         combine_df.to_sql(name= indicator, con=engine, if_exists='replace', index=True)
        time.sleep(0.2)
        print(f'{indicator} retrieved')
    except: 
        print(f'error with {indicator} data. Proceeding...')
        errors.append(indicator)
        
    

USSLIND retrieved
UMCSENT retrieved
CSCICP03USM665S retrieved
INDPRO retrieved
TCU retrieved
DGORDER retrieved
NEWORDER retrieved
PI retrieved
BUSINV retrieved
TOTALSA retrieved
MARTSMPCSM44X72USS retrieved
PAYEMS retrieved
UNRATE retrieved
ICSA retrieved
CCSA retrieved
CC4WSA retrieved
NPPTTL retrieved
CES0500000003 retrieved
HSN1F retrieved
EXHOSLUSM495S retrieved
PERMIT retrieved
CSUSHPINSA retrieved
SP500 retrieved



In [39]:
combine_df.head()

,date,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,2010-03-08,1138.50,0.00,SP500,2020-03-09,2020-03-09,S&P 500,2010-03-08,2020-03-06,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-06 19:10:58-06,85,The observations for the S&P 500 represent the...
1,2010-03-09,1140.45,1.95,SP500,2020-03-09,2020-03-09,S&P 500,2010-03-08,2020-03-06,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-06 19:10:58-06,85,The observations for the S&P 500 represent the...
2,2010-03-10,1145.61,5.16,SP500,2020-03-09,2020-03-09,S&P 500,2010-03-08,2020-03-06,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-06 19:10:58-06,85,The observations for the S&P 500 represent the...
3,2010-03-11,1150.24,4.63,SP500,2020-03-09,2020-03-09,S&P 500,2010-03-08,2020-03-06,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-06 19:10:58-06,85,The observations for the S&P 500 represent the...
4,2010-03-12,1149.99,-0.25,SP500,2020-03-09,2020-03-09,S&P 500,2010-03-08,2020-03-06,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-06 19:10:58-06,85,The observations for the S&P 500 represent the...


## fredapi Examples
s = fred.get_series('SP500', observation_start='1/31/2014')
info = fred.get_series_info('PAYEMS')
info['title']
df = fred.search_by_category(101, limit=10, order_by='popularity', sort_order='desc')
df['title']
more info at https://mortada.net/python-api-for-fred.html

In [7]:
#create spx dataframe

#get data and info series
data =  fred.get_series('SP500', observation_start='3/08/2010')
info = fred.get_series_info('SP500')
data_df = pd.DataFrame(data)
data_df.reset_index(level=0, inplace=True)
info_df = pd.DataFrame(info)
info_df.reset_index(level=0, inplace=True)
#pivot info_df
df_dict = dict(zip(info_df['index'],info_df[0]))
info_df = pd.DataFrame(df_dict,  index=[0])
#duplicate info_df rows to equal data_df length
info_df = pd.concat([info_df]*(len(data_df)), ignore_index=True)
#join df's
spx_df = data_df.join(info_df)
#rename columns
spx_df.rename(columns = {'index':'date', 0:'data'}, inplace = True) 
#create new 'change from last column'
spx_df.insert(2, 'change', np.nan)
#fill nan with zeroes
spx_df['change'].fillna(0, inplace=True)
#create data for 'change' column
changes = [spx_df.iloc[i,1]-spx_df.iloc[i-1,1] for i in range (1,len(spx_df))]
change_clean = [float("{0:.3f}".format(x)) for x in changes]
spx_df['change'][1:len(spx_df)] = change_clean


In [8]:
spx_df.head()

,date,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,2010-03-29,1173.22,0.00,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
1,2010-03-30,1173.27,0.05,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2,2010-03-31,1169.43,-3.84,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
3,2010-04-01,1178.10,8.67,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
4,2010-04-02,NaN,NaN,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...


In [9]:

data = fred.get_series('PAYEMS', observation_start='3/29/2010')
info = fred.get_series_info('PAYEMS')
#store as df's and reset indexes 
data_df = pd.DataFrame(data)
data_df.reset_index(level=0, inplace=True)
info_df = pd.DataFrame(info)
info_df.reset_index(level=0, inplace=True)
#pivot info_df
df_dict = dict(zip(info_df['index'],info_df[0]))
info_df = pd.DataFrame(df_dict,  index=[0])
#duplicate info_df rows to equal data_df length
info_df = pd.concat([info_df]*(len(data_df)), ignore_index=True)
#join df's
combine_df = data_df.join(info_df)
#rename columns
combine_df.rename(columns = {'index':'date', 0:'data'}, inplace = True) 
#create new 'change from last column'
combine_df.insert(2, 'change', np.nan)
#fill nan with zeroes
combine_df['change'].fillna(0, inplace=True)
#create data for 'change' column
changes = [combine_df.iloc[i,1]-combine_df.iloc[i-1,1] for i in range (1,len(combine_df))]
change_clean = [float("{0:.3f}".format(x)) for x in changes]
combine_df['change'][1:len(combine_df)] = change_clean


In [44]:
combine_df.head()

,date,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,2010-03-01,129879.0,0.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
1,2010-04-01,130110.0,231.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
2,2010-05-01,130650.0,540.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
3,2010-06-01,130511.0,-139.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
4,2010-07-01,130427.0,-84.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."


In [11]:
In [21]: pd.DatetimeIndex(df.date).to_period('M')
Out[21]:
<class 'pandas.tseries.period.PeriodIndex'>
[2013-12, ..., 2013-11]
Length: 7, Freq: M

In [22]: df.groupby(pd.DatetimeIndex(df.date).to_period('M')).nth(0)
Out[22]:
    n       date     val
0  17 2013-12-05  0.1621
4  21 2013-11-29  0.1480

In [15]:
spx_df.groupby(pd.DatetimeIndex(spx_df.date).to_period('M')).nth(-1)

,date,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
date,,,,,,,,,,,,,,,,,,
2010-03,2010-03-31,1169.43,-3.84,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2010-04,2010-04-30,1186.69,-20.09,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2010-05,2010-05-31,NaN,NaN,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2010-06,2010-06-30,1030.71,-10.53,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2010-07,2010-07-30,1101.60,0.07,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11,2019-11-29,3140.98,NaN,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2019-12,2019-12-31,3230.78,9.49,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2020-01,2020-01-31,3225.52,-58.14,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...


In [37]:
#group SPX by monthly close
spx_month_df = spx_df.groupby(pd.DatetimeIndex(spx_df.date).to_period('M')).nth(-1)

In [24]:
spx_month_df.columns.values

array(['date', 'data', 'change', 'id', 'realtime_start', 'realtime_end',
       'title', 'observation_start', 'observation_end', 'frequency',
       'frequency_short', 'units', 'units_short', 'seasonal_adjustment',
       'seasonal_adjustment_short', 'last_updated', 'popularity', 'notes'],
      dtype=object)

In [38]:
#rename date col
spx_month_df.rename(columns = {'date':'day'}, inplace = True) 
# spx_month_df.drop(columns = ['date'], inplace=True)


In [39]:
spx_month_df.reset_index(level=0, inplace=True)

In [40]:
spx_month_df.head()

,date,day,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,2010-03,2010-03-31,1169.43,-3.84,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
1,2010-04,2010-04-30,1186.69,-20.09,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
2,2010-05,2010-05-31,NaN,NaN,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
3,2010-06,2010-06-30,1030.71,-10.53,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...
4,2010-07,2010-07-30,1101.60,0.07,SP500,2020-03-28,2020-03-28,S&P 500,2010-03-29,2020-03-27,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2020-03-27 19:10:58-05,87,The observations for the S&P 500 represent the...


In [45]:
combine_month_df = combine_df.groupby(pd.DatetimeIndex(combine_df.date).to_period('M')).nth(-1)
combine_month_df.rename(columns = {'date':'day'}, inplace = True)
combine_month_df.reset_index(level=0, inplace=True)
combine_month_df.head()

,date,day,data,change,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
0,2010-03,2010-03-01,129879.0,0.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
1,2010-04,2010-04-01,130110.0,231.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
2,2010-05,2010-05-01,130650.0,540.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
3,2010-06,2010-06-01,130511.0,-139.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."
4,2010-07,2010-07-01,130427.0,-84.0,PAYEMS,2020-03-28,2020-03-28,"All Employees, Total Nonfarm",1939-01-01,2020-02-01,Monthly,M,Thousands of Persons,Thous. of Persons,Seasonally Adjusted,SA,2020-03-06 07:48:46-06,86,"All Employees: Total Nonfarm, commonly known a..."


In [ ]:
plt.figure(figsize=(10,6))
plt.plot(spx_month_df['date'],spx_month_df['data'], color = "blue", label="S&P 500")
plt.plot(spx_month_df['date'],combine_month_df['change'], color = "red", label="NFP")
plt.title("SPX vs NF", fontsize=20)
plt.xlabel("Star Rating", fontsize=16)
plt.ylabel("Average", fontsize=16)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(loc="best", fontsize=16)
plt.savefig("images/game_total_help_ratio.png")
